In [ ]:
from smolagents import CodeAgent, LiteLLMModel
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

### Code Agent

In [ ]:
model = LiteLLMModel(model_id="openai/gpt-4.1-mini") # Could use 'gpt-4o'
code_agent = CodeAgent(tools=[], model=model, add_base_tools=True)

In [ ]:
code_agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

In [ ]:
code_agent.run(
    "Could you get me the title of the page at url 'https://huggingface.co/blog'?",
)

In [ ]:
code_agent.memory.get_full_steps()
code_agent.memory.steps[2].model_input_messages

#code_agent.logs[2]
#pprint.pprint(code_agent.logs[0].system_prompt)

### Tool Agent

In [ ]:
from smolagents import ToolCallingAgent

tool_agent = ToolCallingAgent(tools=[], model=model, add_base_tools=True)
tool_agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")

### MCP Agent

In [ ]:
from smolagents import ToolCollection, CodeAgent
from mcp import StdioServerParameters
server_parameters = StdioServerParameters(
    command="npx",
    args=["@gongrzhe/server-gmail-autoauth-mcp"],
    env={"UV_PYTHON": "3.13", **os.environ},
)

with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], add_base_tools=True, model=model)
    agent.run("""Help me to find email send by TTBank in last 3 days, ("""
              """if you can find it, reply thanks to TTBank!""")

In [ ]:
agent.logs[4].model_input_messages

In [ ]:
with ToolCollection.from_mcp({"url": "http://127.0.0.1:8000/sse"}, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], add_base_tools=True)
    agent.run("Please find a remedy for hangover.")